In [1]:
# install beautifulsoup for the first time if not there
#!conda install BeautifulSoup4
#!pip install BeautifulSoup4
#!pip install html5lib

In [2]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

/var/folders/h4/gc9911bs2sq4x08kxr_589rw0000gp/T/ipykernel_80100/3244935490.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
html="""<html>
    <head>
        <title>
            This is an example page
        </title>
    </head>
    <body>
        Here is the content of the page
        <p>
            One paragraph
        </p>
        <li> one </li>
        <li> two </li>
        <table>
            <tr>
                <td>col 1</td>
                <td>col 2</td>
            </tr>
            <tr>
                <td>val 1</td>
                <td>val 2</td>
            </tr>
        </table>
        End of body
    </body>
</html>"""

In [4]:
HTML(html)

col 1,col 2
val 1,val 2


In [5]:
root = ET.fromstring(html)

In [6]:
for element in root.iter('td'):
    print(element.text)

col 1
col 2
val 1
val 2


In [7]:
messed_html = html + "blablabla"
root = ET.fromstring(messed_html)

ParseError: junk after document element: line 26, column 7 (<string>)

In [9]:
soup = BeautifulSoup(messed_html, 'html')

In [10]:
for element in soup.find_all('td'):
    print(element.text)

col 1
col 2
val 1
val 2


In [11]:
def print_html_tree(tag,level = 0):
    print('\t'*level + tag.name)
    for child in tag.findChildren():
        print_html_tree(child, level+1)

In [12]:
print_html_tree(soup.currentTag)

[document]
	html
		head
			title
		title
		body
			p
			li
			li
			table
				tr
					td
					td
				td
				td
				tr
					td
					td
				td
				td
			tr
				td
				td
			td
			td
			tr
				td
				td
			td
			td
		p
		li
		li
		table
			tr
				td
				td
			td
			td
			tr
				td
				td
			td
			td
		tr
			td
			td
		td
		td
		tr
			td
			td
		td
		td
	head
		title
	title
	body
		p
		li
		li
		table
			tr
				td
				td
			td
			td
			tr
				td
				td
			td
			td
		tr
			td
			td
		td
		td
		tr
			td
			td
		td
		td
	p
	li
	li
	table
		tr
			td
			td
		td
		td
		tr
			td
			td
		td
		td
	tr
		td
		td
	td
	td
	tr
		td
		td
	td
	td


Extract names and titles of [MIS Faulty at UGA](https://www.terry.uga.edu/directory/?group=management-information-systems)

How many assistant professors are there?

In [20]:
r = requests.get("https://www.terry.uga.edu/directory/?group=management-information-systems")

In [21]:
r.status_code

200

In [22]:
html = r.text

In [23]:
print(html)

<!doctype html>
<html lang="en-US">
<head>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<link rel="profile" href="https://gmpg.org/xfn/11">

	<meta name='robots' content='index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1' />

<!-- Google Tag Manager for WordPress by gtm4wp.com -->
<script data-cfasync="false" data-pagespeed-no-defer>
	var gtm4wp_datalayer_name = "dataLayer";
	var dataLayer = dataLayer || [];
</script>
<!-- End Google Tag Manager for WordPress by gtm4wp.com -->
	<!-- This site is optimized with the Yoast SEO plugin v21.8 - https://yoast.com/wordpress/plugins/seo/ -->
	<title>Directory &#8211; Terry College of Business</title>
	<link rel="canonical" href="https://www.terry.uga.edu/group/management-information-systems/" />
	<meta property="og:locale" content="en_US" />
	<meta property="og:type" content="website" />
	<meta property="og:url" content="https://www.terry.uga.edu/group/management-infor

In [24]:
HTML(html)

In [25]:
root = ET.fromstring(html)

ParseError: syntax error: line 1, column 0 (<string>)

In [26]:
soup = BeautifulSoup(html, 'html')

In [27]:

names = soup.find_all('a', attrs={'class':'entry-title'}, )
names = [name for name in names if name.text != ""]
urls = [name.attrs['href'] for name in names]
names = [name.text for name in names]
len(names)

47

In [28]:
positions = soup.find_all('span', attrs={'class':'position__title'})
len(positions)

49

In [ ]:
# lengths of names and postisions do not match, need to find another way

In [29]:
cards = soup.find_all('div', attrs={'class':'col-12 col-sm-8'})
len(cards)

47

In [30]:
people = []
for card in cards:
    name=card.find('a', attrs={'class':                          
                         'entry-title'}).text
    position=card.find('span', attrs={'class':
                         'position__title'}).text
    person = {'name':name, 'position':position}
    people.append(person)
#people
df = pd.DataFrame(people)

In [31]:
df.head()

,name,position
0,Karen Aguar,Lecturer
1,Mariana Andrade,Assistant Professor
2,Marie-Claude (Maric) Boudreau,Associate Professor
3,Stacy Boyles,Adjunct Lecturer
4,Dave Chatterjee,Associate Professor


In [33]:
sum(df.position == 'Assistant Professor')

7

## using pandas read_html

The web page has to have a table

In [34]:
response = requests.get("https://sicemdawgs.com/uga-football-roster/")
response.status_code

200

In [35]:
dfs = pd.read_html(response.text)

In [36]:
type(dfs)

list

In [37]:
len(dfs)

1

In [38]:
df1 = dfs[0]
df1

,No.,Name,Pos.,Cl.,Ht.,Wt.,Hometown (High School)
0,0,Troy Bowles,ILB,So.,6-0,220,"Tampa, FL (Jesuit)"
1,0,Roderick Robinson II,RB,So.,6-0,240,"San Diego, CA (Lincoln)"
2,2,"Smael Mondon, Jr.",ILB,Sr.,6-3,225,"Dallas, GA (Paulding County)"
3,3,Andrew Paul,RB,RSo.,5-11,218,"Dallas, TX (Parish Episcopal School)"
4,4,Oscar Delp,TE,Jr.,6-5,245,"Cumming, GA (West Forsyth)"
...,...,...,...,...,...,...,...
117,—,Henry Waller,DB,RSo.,5-11,170,"Evans, GA (W.S. of Augusta)"
118,—,Sacovie White,WR,Fr.,5-9,180,"Cartersville, GA (Cass)"
119,—,Justin Williams,ILB,Fr.,6-2,215,"Conroe, TX (Oak Ridge)"
120,—,Colbie Young,WR,Sr.,6-3,215,"Binghamton, NY (Binghamton)"
